In [272]:
import os 
import time 
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import io
from io import BytesIO




In [63]:
DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [219]:
DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    
search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
    
wd.get(search_url.format(q='tree'))
body = wd.find_element_by_tag_name("body")
   
main = []
# iterating past 49 elements after this works of creat
    # a variable that holds
body.send_keys(Keys.PAGE_DOWN)
data = wd.find_elements_by_class_name("rg_i.Q4LuWd")


In [220]:
links = [data[i].get_attribute('src') for i in range(len(data))]

In [224]:
links1 = [data[i].get_attribute('data-src') for i in range(len(data))]

In [273]:
from PIL import Image
import base64
from io import BytesIO
import selenium 
from selenium.webdriver.common.keys import Keys
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator





def scrape(search_name,web_driver=wd):
    DRIVER_PATH = '/Users/trey/Image-Scraping/scraping/chromedriver'
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    
    search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
    
    wd.get(search_url.format(q=search_name))
    body = wd.find_element_by_tag_name("body")
   
    main = []
    for i in range(1):
        body.send_keys(Keys.PAGE_DOWN)
        data = wd.find_elements_by_class_name("rg_i.Q4LuWd")
        #links = data.get_attribute('src')
        #main.append(data)#.get_attribute("src")
        #time.sleep(1.75)
    #for i in range(1
        
    #click = wd.find_element_by_class_name("YstHxe")
    #click.click()
    #time.sleep(5)
    
    #for i in range(1):
    #    body.send_keys(Keys.PAGE_DOWN)
        #main.append(wd.find_elements_by_class_name("rg_i.Q4LuWd"))
    #    time.sleep(1.75)
    #time.sleep(2.5)
    #main = wd.find_elements_by_class_name("rg_i.Q4LuWd")
    return links
    wd.quit()


def sort(main):
    links = [main[i].get_attribute('src') for i in range(len(main))]
    beginning = []
    for i in main:
        beginning_src_link = beginning.append(i[0:6])
    return beginning
        
    
        
    

    
    
    
    
def correct(mains):
    imagess = []
    for image in images:
        if image[0:4] == 'data':
            new = image.replace("data:image/jpeg;base64,","")
            if new[-2:] != '==':
                new_edit = new + '=='
                image = (Image.open(BytesIO(base64.b64decode(new_edit)))).resize((150,150))
                imagess.append(image)
            else:
                image = (Image.open(BytesIO(base64.b64decode(new)))).resize((150,150))
                imagess.append(image)
        if image[0:4] == 'http':
            new = requests.get(image)
            image = Image.open(BytesIO(image.content))
            imagess.append(image)
    return images





def save(images):
    os.mkdir('Images')
    os.mkdir('Images/regular')
    index = 0 
    for i in images:
        i.save('Images/regular'+'/'+str(index)+'.jpeg')
        index+=1
       
    
    
    
    
#def augment(**kwargs):
    #data_gen = ImageDataGenerator(
 #                           rotation_range = 30,
  #                          width_shift_range=.3,
   #                         height_shift_range=.4,
    #                         shear_range=30,
     #                        horizontal_flip=True,
      #                       vertical_flip=True,
       #                      zoom_range=0.2)
    
    
    
    
    
    
    
def augment(images):    
    os.mkdir('Images/augmented')
    for image in images:
        i = 0 
        image = np.expand_dims(image,axis=0)
        for batch in data_gen.flow(image, 
                           batch_size=1,
                          save_to_dir='Images/augmented', 
                           save_prefix='tree', 
                           save_format='jpeg'):
            i += 1 
            if i > 20:
                break

    
   


In [164]:
imagess = []
for image in images:
    if image[0:4] == 'data':
        new = image.replace("data:image/jpeg;base64,","")
        if new[-2:] != '==':
            new_edit = new + '=='
            image = (Image.open(BytesIO(base64.b64decode(new_edit)))).resize((150,150))
            imagess.append(image)
        else:
            image = (Image.open(BytesIO(base64.b64decode(new)))).resize((150,150))
            imagess.append(image)
    if image[0:4] == 'http':
        new = requests.get(image)
        image = Image.open(BytesIO(image.content))
        imagess.append(image)
        
        


In [ ]:

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#model.save_weights('first_try.h5')

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)